In [1]:
import polars as pl
from pathlib import Path
import os

In [2]:
site_to_acccession_map = {
    "CCH_0001_WB_01": "EB100001",
    "CCH_0002_WB_01": "EB100002",
    "CCH_0003_WB_01": "EB100003",
    "CKD_0001_WB_01": "EB100004",
    "CKD_0002_WB_01": "EB100005",
    "CKD_0003_WB_01": "EB100006",
    "EXP_0001_WB_01": "EB100007",
    "EXP_0002_WB_01": "EB100008",
    "EXP_0003_WB_01": "EB100009",
    "MOM_0001_WB_01": "EB100010",
    "MOM_0002_WB_01": "EB100011",
    "MOM_0003_WB_01": "EB100012",
    "UIC_0004_WB_01": "EB100013",
    "UIC_0005_WB_01": "EB100014",
    "UIC_0006_WB_01": "EB100015"
}

site_to_acccession_map = {v: k for k, v in site_to_acccession_map.items()}

print(site_to_acccession_map)

{'EB100001': 'CCH_0001_WB_01', 'EB100002': 'CCH_0002_WB_01', 'EB100003': 'CCH_0003_WB_01', 'EB100004': 'CKD_0001_WB_01', 'EB100005': 'CKD_0002_WB_01', 'EB100006': 'CKD_0003_WB_01', 'EB100007': 'EXP_0001_WB_01', 'EB100008': 'EXP_0002_WB_01', 'EB100009': 'EXP_0003_WB_01', 'EB100010': 'MOM_0001_WB_01', 'EB100011': 'MOM_0002_WB_01', 'EB100012': 'MOM_0003_WB_01', 'EB100013': 'UIC_0004_WB_01', 'EB100014': 'UIC_0005_WB_01', 'EB100015': 'UIC_0006_WB_01'}


In [8]:
fastq_files = sorted(list(Path("/zata/zippy/ramirezc/gembs-smk/.test3").glob("*.fastq.gz")))
fastq_files = [str(file) for file in fastq_files]

print(fastq_files)

records = []
for file in fastq_files:
    basename = os.path.basename(file)
    parts = basename.split("_")
    record = {
        "Project": parts[0],
        "Assay": "WGBS",
        "Barcode": parts[1],
        "Dataset": site_to_acccession_map[parts[1]],
        "Read": parts[2].split(".")[0],
        "File": file
    }
    records.append(record)

print(records)
    
wgbs_metadata = pl.from_dicts(records).pivot(
        index=["Barcode", "Project", "Assay", "Dataset"],
        on="Read", 
        values="File",
        maintain_order=True
    ).rename({"R1": "File1", "R2": "File2", }).drop("Project", "Assay")

wgbs_metadata.write_csv("/zata/zippy/ramirezc/gembs-smk/notebooks/gembs_metadata.csv")
print(wgbs_metadata)

['/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100001_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100001_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100002_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100002_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100003_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100003_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100004_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100004_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100005_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100005_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100006_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100006_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100007_R1.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100007_R2.fastq.gz', '/zata/zippy/ramirezc/gembs-smk/.test3/MOHD_EB100008_R1.fastq